In [14]:
import netket as nk
import netket.experimental as nkx
import netket.nn as nknn

from math import pi

from netket.experimental.operator.fermion import destroy as c
from netket.experimental.operator.fermion import create as cdag
from netket.experimental.operator.fermion import number as nc

from vmc_torch.fermion_utils import generate_random_fpeps, product_bra_state, get_amp
import quimb.tensor as qtn
from autoray import do
import symmray as sr
import pickle
import os
os.environ['NUMBA_NUM_THREADS'] = '20'

# Define the lattice shape
L = 4  # Side of the square
Lx = int(L/2)
Ly = int(L/2)
# graph = nk.graph.Square(L)
graph = nk.graph.Grid([Lx,Ly], pbc=False)
N = graph.n_nodes


# Define the fermion filling and the Hilbert space
# N_f = int(Lx*Ly/2)
N_f = 3
hi = nkx.hilbert.SpinOrbitalFermions(N, s=1/2, n_fermions=N_f)
# hi = nkx.hilbert.SpinOrbitalFermions(N, s=1/2, n_fermions_per_spin=(N_f//2,N_f//2))


# Define the Hubbard Hamiltonian
t = 1.0
U = 8.0
mu = 0.0

H = 0.0
for (i, j) in graph.edges(): # Definition of the Hubbard Hamiltonian
    for spin in (1,-1):
        H -= t * (cdag(hi,i,spin) * c(hi,j,spin) + cdag(hi,j,spin) * c(hi,i,spin))
for i in graph.nodes():
    H += U * nc(hi,i,+1) * nc(hi,i,-1)


# Exact diagonalization of the Hamiltonian for benchmark
sp_h = H.to_sparse() # Convert the Hamiltonian to a sparse matrix
from scipy.sparse.linalg import eigsh
from scipy.linalg import eigh
try:
    eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")
    E_gs = eig_vals[0]
    psi_gs = eig_vecs[:,0]
except:
    eig_val, eig_vec = eigh(sp_h.toarray())
    E_gs = eig_val
    psi_gs = eig_vec[:,0]

print("Exact ground state energy per site:", E_gs/N)

Exact ground state energy per site: -0.5811388300841901


In [15]:
import jax
import numpy as np
random_seed = np.random.randint(2**32 - 1)
config = hi.random_state(key=jax.random.PRNGKey(random_seed))

# --- Utils ---
def from_netket_config_to_quimb_config(netket_configs):
    def func(netket_config):
        """Translate netket spin-1/2 config to tensor network product state config"""
        total_sites = len(netket_config)//2
        spin_up = netket_config[:total_sites]
        spin_down = netket_config[total_sites:]
        sum_spin = spin_up + spin_down
        quimb_config = np.zeros(total_sites, dtype=int)
        for i in range(total_sites):
            if sum_spin[i] == 0:
                quimb_config[i] = 0
            if sum_spin[i] == 2:
                quimb_config[i] = 3
            if sum_spin[i] == 1:
                if spin_down[i] == 1:
                    quimb_config[i] = 1
                else:
                    quimb_config[i] = 2
        return quimb_config
    if len(netket_configs.shape) == 1:
        return func(netket_configs)
    else:
        # batched
        return np.array([func(netket_config) for netket_config in netket_configs])

def from_quimb_config_to_netket_config(quimb_config):
    """Translate tensor network product state config to netket spin-1/2 config"""
    total_sites = len(quimb_config)
    spin_up = np.zeros(total_sites, dtype=int)
    spin_down = np.zeros(total_sites, dtype=int)
    for i in range(total_sites):
        if quimb_config[i] == 0:
            spin_up[i] = 0
            spin_down[i] = 0
        if quimb_config[i] == 1:
            spin_up[i] = 0
            spin_down[i] = 1
        if quimb_config[i] == 2:
            spin_up[i] = 1
            spin_down[i] = 0
        if quimb_config[i] == 3:
            spin_up[i] = 1
            spin_down[i] = 1
    return np.concatenate((spin_up, spin_down))


# Spinful PEPS

In [16]:
D = 4
seed = 2
symmetry = 'U1'
spinless = False
peps = generate_random_fpeps(Lx, Ly, D=4, seed=2, symmetry=symmetry, Nf=N_f, spinless=spinless)[0]
edges = qtn.edges_2d_square(Lx, Ly, cyclic=False)
site_info = sr.utils.parse_edges_to_site_info(
    edges,
    D,
    phys_dim=4,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

terms = {
    (sitea, siteb): sr.fermi_hubbard_local_array(
        t=t, U=U, mu=mu,
        symmetry=symmetry,
        coordinations=(
            site_info[sitea]['coordination'],
            site_info[siteb]['coordination'],
        ),
    ).fuse((0, 1), (2, 3))
    for (sitea, siteb) in peps.gen_bond_coos()
}
ham = qtn.LocalHam2D(Lx, Ly, terms)

su = qtn.SimpleUpdateGen(peps, ham, compute_energy_per_site=True,D=D, compute_energy_opts={"max_distance":2}, gate_opts={'cutoff':1e-10})
su.evolve(25, tau=0.3)
peps = su.state
gs = su.get_state()

n=25, tau=0.3000, energy~-0.531665: 100%|##########| 25/25 [00:00<00:00, 71.92it/s]


In [17]:
import numpy as np
def product_bra_state(psi, config, check=False,reverse=False, dualness=False):
    product_tn = qtn.TensorNetwork()
    backend = psi.tensors[0].data.backend
    dtype = eval(backend+'.'+psi.tensors[0].data.dtype)
    if psi.spinless:
        index_map = {0: 0, 1: 1}
        array_map = {
            0: do('array',[1.0],like=backend,dtype=dtype), 
            1: do('array',[1.0],like=backend,dtype=dtype)
        }
    else:
        if psi.symmetry == 'Z2':
            index_map = {0:0, 1:1, 2:1, 3:0}
            array_map = {
                0: do('array',[1.0, 0.0],like=backend,dtype=dtype), 
                1: do('array',[1.0, 0.0],like=backend,dtype=dtype), 
                2: do('array',[0.0, 1.0],like=backend,dtype=dtype), 
                3: do('array',[0.0, 1.0],like=backend,dtype=dtype)
            }
        elif psi.symmetry == 'U1':
            index_map = {0:0, 1:1, 2:1, 3:2}
            array_map = {
                0: do('array',[1.0],like=backend,dtype=dtype), 
                1: do('array',[1.0, 0.0],like=backend,dtype=dtype), 
                2: do('array',[0.0, 1.0],like=backend,dtype=dtype), 
                3: do('array',[1.0],like=backend,dtype=dtype)
            }
    iter = zip(config, psi.sites) if not reverse else zip(config[::-1], psi.sites[::-1])
    for n, site in iter:
        p_ind = psi.site_ind_id.format(*site)
        p_tag = psi.site_tag_id.format(*site)
        tid = psi.sites.index(site)

        n_charge = index_map[int(n)]
        n_array = array_map[int(n)]

        oddpos = None
        if not psi.spinless:
            if int(n) == 1:
                oddpos = (3*tid+1)*(-1)#**reverse
            elif int(n) == 2:
                oddpos = (3*tid+2)*(-1)#**reverse
            elif int(n) == 3:
                # oddpos = ((3*tid+1)*(-1)**reverse, (3*tid+2)*(-1)**reverse)
                oddpos = None
        else:
            oddpos = (3*tid+1)*(-1)**reverse
        
        tsr_data = sr.FermionicArray.from_blocks(
            blocks={(n_charge,):n_array}, 
            duals=(dualness,),
            symmetry=psi.symmetry, 
            charge=n_charge, 
            oddpos=oddpos
        )
        if check:
            if int(n)==0:
                blocks = {(0,): do('array',[1.0],like=backend,dtype=dtype), (1,): do('array',[0.0, 0.0],like=backend,dtype=dtype), (2,):do('array',[0.0],like=backend,dtype=dtype)}
            elif int(n)==1:
                blocks = {(0,): do('array',[0.0],like=backend,dtype=dtype), (1,): do('array',[1.0, 0.0],like=backend,dtype=dtype), (2,):do('array',[0.0],like=backend,dtype=dtype)}
            elif int(n)==2:
                blocks = {(0,): do('array',[0.0],like=backend,dtype=dtype), (1,): do('array',[0.0, 1.0],like=backend,dtype=dtype), (2,):do('array',[0.0],like=backend,dtype=dtype)}
            elif int(n)==3:
                blocks = {(0,): do('array',[0.0],like=backend,dtype=dtype), (1,): do('array',[0.0, 0.0],like=backend,dtype=dtype), (2,):do('array',[1.0],like=backend,dtype=dtype)}
            tsr_data = sr.FermionicArray.from_blocks(
                blocks=blocks, 
                duals=(dualness,),
                symmetry=psi.symmetry, 
                charge=n_charge, 
                oddpos=oddpos
            )
        tsr = qtn.Tensor(data=tsr_data, inds=(p_ind,),tags=(p_tag, 'bra', f'X{site[0]}', f'Y{site[1]}'))
        product_tn |= tsr

    product_tn.view_as_(
        qtn.PEPS,
        site_ind_id="k{},{}",
        site_tag_id="I{},{}",
        x_tag_id="X{}",
        y_tag_id="Y{}",
        Lx=psi.Lx,
        Ly=psi.Ly,
    )
    return product_tn

In [18]:
from symmray.symmetries import  calc_phase_permutation
import numpy
all_configs = hi.all_states()
if not spinless:
    # all_configs = [from_netket_config_to_quimb_config(config) for config in all_configs]
    all_configs = from_netket_config_to_quimb_config(all_configs)
phase_list = []
axes = tuple(range(peps.nsites - 1, -1, -1))
parity_map = {0:0, 1:1, 2:1, 3:0}

for config in all_configs:
    config_parity = np.array([parity_map[config[i]] for i in range(len(config))])
    perm_phase = calc_phase_permutation(tuple(config_parity), axes)
    # print(config, perm_phase, config_parity)
    phase_list.append(perm_phase)

ket_list = [product_bra_state(peps, config, check=True, reverse=False, dualness=False) for config in all_configs]
bra1_list = [product_bra_state(peps, config, check=True, reverse=True, dualness=True) for config in all_configs]
bra_list = [product_bra_state(peps, config, check=True, reverse=True, dualness=False).conj() for config in all_configs]
hamiltonian = H.to_dense()

In [19]:
def check_oddpos(tn):
    oddpos_list = []
    try:
        for array in tn.arrays:
            oddpos_list.append(array.oddpos)
        return oddpos_list
    except:
        oddpos_list.append(tn.data.oddpos)
        return oddpos_list

In [20]:
from symmray.symmetries import  calc_phase_permutation
from vmc_torch.fermion_utils import get_spinful_charge_map
import itertools

def calc_phase_hopping(where, config):
    site1, site2 = where
    site_ind1 = site1[0]*Ly + site1[1]
    site_ind2 = site2[0]*Ly + site2[1]
    parity_map = {0:0, 1:1, 2:1, 3:2}
    parity_config = np.array([parity_map[config[i]] for i in range(len(config))])
    if sum(parity_config[site_ind1+1:site_ind2])%2 == 0:
        return 1
    else:
        return -1
def calc_reverse_phase(config):
    axes = tuple(range(len(config) - 1, -1, -1))
    parity_map = {0:0, 1:1, 2:1, 3:0}
    config_parity = np.array([parity_map[config[i]] for i in range(len(config))])
    perm_phase = calc_phase_permutation(tuple(config_parity), axes)
    perm_phase *= (-1)**np.count_nonzero(np.array(config)==3)
    return perm_phase

In [21]:
new_ham = np.zeros_like(hamiltonian)
raw_matrix = np.zeros_like(hamiltonian)
for i, j in itertools.product(range(len(bra_list)), repeat=2):
    if hamiltonian[i,j] == 0:
        continue
    bra = bra_list[i]
    ket = bra_list[j].conj()
    config = all_configs[j]
    matrix_element = 0
    for where, G in ham.terms.items():
        x = (bra|ket.gate(G, where=where, contract=True)).contract()
        if x!=0:
            raw_matrix[i,j] += x
            # x *= calc_reverse_phase(config)
            if i!=j:
                x *= calc_phase_hopping(where, config)
            matrix_element += x
    new_ham[i,j] = matrix_element * (-1)**N_f

# T--->---: Dual, ind-
# T---<---: NOT dual, ind+

# T1---<---T2: no parity factor, +-
# T1--->---T2: has parity factor, -+

In [22]:
# Netket Hamiltonian energy check
import numpy as np
all_configs = hi.all_states()
if not spinless:
    # all_configs = [from_netket_config_to_quimb_config(config) for config in all_configs]
    all_configs = from_netket_config_to_quimb_config(all_configs)

# psi_vec = np.array([gs.get_amp(config).contract() for config in all_configs])
psi_vec = np.array([(bra|gs).contract() for bra in bra_list])
psi_vec_ket = np.array([(ket.conj()|gs).contract() for ket in ket_list])
psi_vec = psi_vec/np.linalg.norm(psi_vec)
psi_vec_ket = psi_vec_ket/np.linalg.norm(psi_vec_ket)

print((psi_vec.conj().T @ (new_ham @ psi_vec))/N, (psi_vec.conj().T @ (hamiltonian @ psi_vec))/N, (psi_vec.conj().T @ (raw_matrix @ psi_vec))/N)

-0.5316648525122848 -0.20381149290496242 -0.0557336230549691


In [23]:
def detect_hopping(configi, configj):
    site_ls =  (configi-configj).nonzero()[0]
    if len(site_ls) == 2:
        return site_ls
    else:
        return None

def calc_reverse_phase(config):
    axes = tuple(range(len(config) - 1, -1, -1))
    parity_map = {0:0, 1:1, 2:1, 3:0}
    config_parity = np.array([parity_map[config[i]] for i in range(len(config))])
    perm_phase = calc_phase_permutation(tuple(config_parity), axes)
    perm_phase *= (-1)**np.count_nonzero(np.array(config)==3)
    return perm_phase


def detect_hopping(configi, configj):
    site_ls =  (configi-configj).nonzero()[0]
    if len(site_ls) == 2:
        return site_ls
    else:
        return None

def calc_phase_netket(configi, configj):
    """Calculate the phase factor for the matrix element in the netket basis, where all spin-up spins are placed before all spin-down spins"""
    hopping = detect_hopping(configi, configj)
    if hopping is not None:
        netket_config_i = from_quimb_config_to_netket_config(configi)
        netket_config_j = from_quimb_config_to_netket_config(configj)
        netket_site_i, netket_site_j = (netket_config_i - netket_config_j).nonzero()[0]
        phase = (-1)**(sum(netket_config_i[netket_site_i+1:netket_site_j])%2)
        return phase
    else:
        return 1

def calc_phase_symmray(configi, configj):
    """Calculate the operator matrix element phase in symmray, assuming local basis convention for spinful fermion is (|up, down>).
    Globally, this convention gives a staggered spin configuration (|dududu...>) when the configuration is flattened to 1D."""
    hopping = detect_hopping(configi, configj)
    if hopping is not None:
        netket_config_i = from_quimb_config_to_netket_config(configi)
        netket_config_j = from_quimb_config_to_netket_config(configj)
        config_i_spin_up =netket_config_i[:len(netket_config_i)//2]
        config_i_spin_down =netket_config_i[len(netket_config_i)//2:]
        config_j_spin_up =netket_config_j[:len(netket_config_j)//2]
        config_j_spin_down =netket_config_j[len(netket_config_j)//2:]
        symmray_config_i = tuple()
        symmray_config_j = tuple()
        for i in range(len(config_i_spin_up)):
            symmray_config_i += (config_i_spin_down[i], config_i_spin_up[i])
            symmray_config_j += (config_j_spin_down[i], config_j_spin_up[i])
        symmray_site_i, symmray_site_j = (np.asarray(symmray_config_i) - np.asarray(symmray_config_j)).nonzero()[0]
        phase = (-1)**(sum(symmray_config_i[symmray_site_i+1:symmray_site_j])%2)
        return phase
    else:
        return 1

def calc_phase_correction_netket_symmray(configi, configj):
    phase_netket = calc_phase_netket(configi, configj)
    phase_symmray = calc_phase_symmray(configi, configj)
    return phase_netket/phase_symmray

In [27]:
# i_ls, j_ls = (new_ham[:20,:20]-hamiltonian[:20,:20]).nonzero()
# i_ls, j_ls = range(20), range(20)
new_ham1 = np.zeros_like(hamiltonian)
corrected_hamiltonian = np.zeros_like(hamiltonian)
for i, j in itertools.product(range(new_ham.shape[0]), repeat=2):
    if detect_hopping(all_configs[i], all_configs[j]) is not None and hamiltonian[i,j]!=0:
        # print(all_configs[i], all_configs[j])
        config_i, config_j = all_configs[i], all_configs[j]
        site_i, site_j = detect_hopping(all_configs[i], all_configs[j])

        new_ham1[i,j] = calc_phase_symmray(config_i,config_j)*(-t)
        corrected_hamiltonian[i,j] = hamiltonian[i,j]*calc_phase_correction_netket_symmray(config_i, config_j)
    else:
        if i==j:
            # print(new_ham[i,j]==hamiltonian[i,j], new_ham[i,j], hamiltonian[i,j])
            double_occupied_nsite = np.count_nonzero(np.array(all_configs[i])==3)
            # print(double_occupied_nsite)
            new_ham1[i,j] = U*double_occupied_nsite
            corrected_hamiltonian[i,j] = U*double_occupied_nsite

In [28]:
x_list, y_list = (new_ham-new_ham1).nonzero()
for x, y in zip(x_list, y_list):
    print(all_configs[x], all_configs[y], new_ham[x,y], new_ham1[x,y])

In [29]:
psi_vec @ (new_ham1 @ psi_vec)/N, psi_vec @ (corrected_hamiltonian @ psi_vec)/N

(np.float64(-0.5316648525122848), np.float64(-0.5316648525122848))

In [486]:
norm, ket, bra = peps.make_norm(return_all=True)
E_sum = 0
for where, G in ham.terms.items():
    tn = bra|ket.gate(G, where=where, contract=False)
    print(where, tn.contract()/norm.contract())
    E_sum += tn.contract()/norm.contract()
print(E_sum/N)
where, G = list(ham.terms.items())[0]

((0, 0), (0, 1)) -0.6179051591050442
((0, 0), (1, 0)) -0.5085233505259702
((0, 1), (1, 1)) -0.5066834316730553
((1, 0), (1, 1)) -0.4721755616856362
-0.5263218757474265


In [89]:
import itertools
# site 0 -- site 1
where, G = list(ham.terms.items())[0]
new_ham0 = np.zeros_like(hamiltonian)
E_test=0
for i, j in itertools.product(range(len(bra_list)), repeat=2):
    bra = bra_list[i]
    ket = ket_list[j]
    x = (bra|ket.gate(G, where=where, contract=True)).contract()
    if x!=0:
        # print(all_configs[i], all_configs[j], x, hamiltonian[i,j])
        x *= calc_reverse_phase(all_configs[i])
        E_test += x * psi_vec[i].conj() * psi_vec[j]
        new_ham0[i,j] += x
E_test

np.float64(-0.48218401217401274)

In [91]:
# site 0 -- site 2
where, G = list(ham.terms.items())[1]
E_test=0
for i, j in itertools.product(range(len(bra_list)), repeat=2):
    bra = bra_list[i]
    ket = ket_list[j]
    x = (bra|ket.gate(G, where=where, contract=True)).contract()
    if parity_map[all_configs[j][1]]==1 and x!=0 and i!=j:
        x*= -1
    if x!=0:
        # print(all_configs[i], all_configs[j], psi_vec[i].conj(), psi_vec[j], x)
        x *= calc_reverse_phase(all_configs[i])
        E_test += x * psi_vec[i].conj() * psi_vec[j]
        new_ham0[i,j] += x
E_test

np.float64(-0.6672034796311469)

In [109]:
# site 1 -- site 3
where, G = list(ham.terms.items())[2]
E_test=0
for i, j in itertools.product(range(len(bra_list)), repeat=2):
    bra = bra_list[i]
    ket = ket_list[j]
    x = (bra|ket.gate(G, where=where, contract=True)).contract()
    if parity_map[all_configs[j][2]]==1 and x!=0:
        x*= -1
    if x!=0:
        # print(all_configs[i], all_configs[j], psi_vec[i].conj(), psi_vec[j], x)\
        x *= calc_reverse_phase(all_configs[i])
        E_test += x * psi_vec[i].conj() * psi_vec[j]
        new_ham0[i,j] += x
E_test

np.float64(-0.9472856447325668)

In [108]:
# site 2 -- site 3
where, G = list(ham.terms.items())[3]
E_test=0
for i, j in itertools.product(range(len(bra_list)), repeat=2):
    bra = bra_list[i]
    ket = ket_list[j]
    x = (bra|ket.gate(G, where=where, contract=True)).contract()
    if x!=0:
        # print(all_configs[i], all_configs[j], psi_vec[i].conj(), psi_vec[j], x)
        x *= calc_reverse_phase(all_configs[i])
        E_test += x * psi_vec[i].conj() * psi_vec[j]
        new_ham0[i,j] += x
E_test

np.float64(-0.28001258550654573)

In [348]:
psi_vec @ (new_ham0 @ psi_vec)/N

np.float64(-0.5514377150952766)

In [25]:
N_terms = {
    site: sr.fermi_number_operator_spinful_local_array(
        symmetry=symmetry
    )
    for site in peps.gen_site_coos()
}
gs = su.get_state()
gs.compute_local_expectation(
    N_terms, normalized=True, max_bond=64,
)/N, gs.compute_local_expectation(
    ham.terms, normalized=True, max_bond=128,
)/N,

(np.float64(0.12499999999999986), np.float64(-0.3220275415290801))

In [ ]:
from symmray.fermionic_local_operators import  build_local_fermionic_elements, build_local_fermionic_dense, get_spinful_indexmap, build_local_fermionic_array
au = sr.FermionicOperator("au")
ad = sr.FermionicOperator("ad")
basis_a = ((), (ad.dag,), (au.dag,), (au.dag, ad.dag))
bases = (basis_a,)
indexmap = get_spinful_indexmap('U1')
terms = (
    (1, (au.dag,)),
)

build_local_fermionic_elements(terms, bases)

# Energy check

In [ ]:
# Netket Hamiltonian energy check
import numpy as np
all_configs = hi.all_states()
if not spinless:
    # all_configs = [from_netket_config_to_quimb_config(config) for config in all_configs]
    all_configs = from_netket_config_to_quimb_config(all_configs)

psi_vec = np.array([gs.get_amp(config).contract() for config in all_configs])

hamiltonian = H.to_dense()
psi_vec = psi_vec/np.linalg.norm(psi_vec)
print((psi_vec.conj().T @ (hamiltonian @ psi_vec))/N)

In [ ]:
# Quimb energy check
gs.compute_local_expectation(
    ham.terms, normalized=True, max_bond=64,
)/N

In [ ]:
psi_gs, psi_vec

In [ ]:
def to_array_check(peps):
    all_config = hi.all_states()
    if not spinless:
        all_config = [from_netket_config_to_quimb_config(config) for config in all_config]
    psi = np.asarray([peps.get_amp(config, conj=True).contract() for config in all_config])
    # print(psi)
    psi = psi/do('linalg.norm', psi)
    return psi

def compute_energy_check(model, hamiltonian):
    psi_gs = to_array_check(model)
    return psi_gs.conj().T@(hamiltonian@psi_gs)/N

hamiltonian = H

compute_energy_check(peps, hamiltonian)

In [ ]:
n_site = N_terms[(0,0)]
inda, indb = 'a', 'b'
product_state1 = sr.FermionicArray.from_blocks(blocks={(2,):np.array([1])}, duals=(True,),symmetry='U1', charge=2, oddpos=1)
product_state2 = sr.FermionicArray.from_blocks(blocks={(2,):np.array([1])}, duals=(True,),symmetry='U1', charge=2, oddpos=2)
n_site_tensor = qtn.Tensor(data=n_site, inds=(inda,indb), tags=(f'I{inda},{indb}',))
product_state_tensor1 = qtn.Tensor(data=product_state1, inds=(inda,), tags=(f'I{inda}',))
product_state_tensor2 = qtn.Tensor(data=product_state2, inds=(indb,), tags=(f'I{indb}',))
(product_state_tensor1.conj()|n_site_tensor|product_state_tensor2).contract(), (product_state_tensor1|product_state_tensor1.conj()).contract(), (n_site_tensor|product_state_tensor1.conj()).contract().data.to_dense()

# Cyclic PEPS

In [5]:
peps = qtn.PEPS.rand(4, 4, 4, cyclic=True)

In [13]:
# SU in quimb
D = 4
seed = 2
symmetry = 'Z2'
cyclic = True
# peps, parity_config = generate_random_fpeps(Lx, Ly, D, seed, symmetry, Nf=N_f)

edges = qtn.edges_2d_square(Lx, Ly, cyclic=cyclic)
site_info = sr.utils.parse_edges_to_site_info(
    edges,
    D,
    phys_dim=2,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

peps = qtn.TensorNetwork()
parity_config = np.zeros(Lx*Ly, dtype=int)
rng = np.random.default_rng(seed)
for site, info in sorted(site_info.items()):
    tid = site[0] * Ly + site[1]
    # bond index charge distribution
    block_indices = [
        sr.BlockIndex({0: d // 2, 1: d // 2}, dual=dual)
        for d, dual in zip(info["shape"][:-1], info["duals"][:-1])
    ]

    # physical index
    p = info['shape'][-1]
    block_indices.append(
        sr.BlockIndex({0: p // 2, 1: p // 2}, dual=info["duals"][-1])
    )

    data = sr.Z2FermionicArray.random(
        block_indices,
        charge=1 if parity_config[tid] else 0,
        seed=rng,
        oddpos=2*tid,
    )

    peps |= qtn.Tensor(
        data=data,
        inds=info["inds"],
        tags=info["tags"],
    )

# required to view general TN as an actual PEPS
for i, j in site_info:
    peps[f"I{i},{j}"].add_tag([f"X{i}", f"Y{j}"])

peps.view_as_(
    fPEPS,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
    x_tag_id="X{}",
    y_tag_id="Y{}",
    Lx=Lx,
    Ly=Ly,
)
peps = peps.copy() # set symmetry during initialization

In [ ]:
peps.draw()